In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from tqdm import *
import tensorflow as tf
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, AveragePooling1D, Flatten, Dense, Input, BatchNormalization, Activation, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.data import Dataset
from keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import random

In [98]:
#########################################################
############# Reading data & Set labels #################
#########################################################

def trigger_idx(trig_arr):
    trig_arr=np.mean(trig_arr,axis=0)
    start_index=[]


    slice_idxs=[]

    for i in range(trig_arr.shape[0]):
        if i+1<trig_arr.shape[0]:
            if abs(trig_arr[i+1]-trig_arr[i])<50 and trig_arr[i]>3000:
                slice_idxs.append(i)

    start_index.append(slice_idxs[0])
    for i in range(len(slice_idxs)):
        if i+1<len(slice_idxs):
            if slice_idxs[i+1]-slice_idxs[i]>1:
                start_index.append(slice_idxs[i+1])


    start_index=np.array(start_index)
    return start_index



#########################################################
############# Reading data & Set labels #################
#########################################################
def Reading_data_Train(tr_url,meta_url,coef_start,coeff_samples,block_num,start,coeff_i):
    X=[]
    Y=[]
    c=[0]*2
    samples_num=coeff_samples[coeff_i]
    for block in trange(block_num):
        tr_arr=np.load(tr_url.format(block+start))
        meta_arr=np.load(meta_url.format(block+start),allow_pickle=True)
        y_str=meta_arr['allbytes']

        for t in range(tr_arr.shape[0]):
            for pos in range(256):
                y_slice=y_str[t][pos*8:pos*8+8]
                if y_slice == "00000000" and pos%4==coeff_i:
        
                    for i in range(3):
                        offset=random.choice([-1,0,1])
                        coeff_i_idx=pos//4
                        if coef_start[coeff_i_idx]>2:
                            trace_slice=tr_arr[t][coef_start[coeff_i_idx]+offset:coef_start[coeff_i_idx]+offset+samples_num]
                        else:
                            offset=0
                            trace_slice=tr_arr[t][coef_start[coeff_i_idx]+offset:coef_start[coeff_i_idx]+offset+samples_num]

                        X.append(trace_slice)
                        Y.append(0)
                        c[0]+=1
                elif y_slice != "00000000" and pos%4==coeff_i and c[1]<=c[0]*32:
        
                    for i in range(5):
                        offset=random.choice([-2,-1,0,1,2])
                        coeff_i_idx=pos//4
                        if coef_start[coeff_i_idx]>2:
                            trace_slice=tr_arr[t][coef_start[coeff_i_idx]+offset:coef_start[coeff_i_idx]+offset+samples_num]
                        else:
                            offset=0
                            trace_slice=tr_arr[t][coef_start[coeff_i_idx]+offset:coef_start[coeff_i_idx]+offset+samples_num]

                        X.append(trace_slice)
                        Y.append(1)
                        c[1]+=1

    X=np.array(X)
    Y=np.array(Y)
    print("labels numbers: ",c)
    print("X:",X.shape)
    print("Y:",Y.shape)

    Y = tf.keras.utils.to_categorical(Y, num_classes=2,dtype=int)  
    return X,Y




#########################################################
############# Reading data & Set labels #################
#########################################################
# poly_idx: the index of the poly, where idx range from 0 to 2
def Attack(tr_url,meta_url,z_url,models,coef_start,coeff_samples,scalers,block_num,start):
    result=[]
    g=0
    real=0
    right=0
    for block in range(block_num):
        tr_arr=np.load(tr_url.format(block+start))
        meta_arr=np.load(meta_url.format(block+start),allow_pickle=True)
        z_arr=np.load(z_url.format(block+start),allow_pickle=True)
        Z=z_arr['Z']
        poly=meta_arr['index']
        y_str=meta_arr['allbytes']
        
        for t_len in trange(tr_arr.shape[0]):
            # 所有曲线进行遍历
            for pos in range(256):
                y_slice=y_str[t_len][pos*8:pos*8+8]
                Z_val=Z[t_len][poly[t_len]][pos]
                if abs(Z_val)>=78:
                    continue
                else:
                    coeff_type=pos%4
                    coeff_idx=pos//4
                    s=coef_start[coeff_type][coeff_idx]
                    trace_slice=tr_arr[t_len][s:s+coeff_samples[coeff_type]]
                    coeff_model=models[coeff_type]
                    trace_slice=trace_slice.reshape(1,trace_slice.shape[0])
                    trace_slice=scalers[coeff_type].transform(trace_slice)
                    predictions = coeff_model.predict(trace_slice,verbose=0)
                    predicted_classes = np.argmax(predictions, axis=1)
                    if predicted_classes[0] == 0 and abs(Z_val)<40:
                        g+=1
                        result.append(np.array([block,t_len,pos]))
                    if y_slice=="00000000":
                        real+=1
                    if predicted_classes[0] == 0 and abs(Z_val)<40 and y_slice=="00000000":
                        right+=1
    print("Guess: {}, Real: {}, Right: {}".format(g,real,right))

    ans=np.array(result)
    np.save(r"F:\20241203_dilithium_y\result.npy",ans)

    return 



#########################################################
############# AttackTraces Preprocssing #################
#########################################################

def AttackTraces_Pre(tr_url,poly_lb_url,poly_idx):
    trace=[]
    for i in range(1):
        tr_arr=np.load(tr_url.format(i))
        lb=np.load(poly_lb_url.format(i),allow_pickle=True)
        # print(lb)
        for j in range(125):
            if lb[1][j]==poly_idx:
                trace.append(tr_arr[j])
                # print(lb[n])
    return np.array(trace)



def shuffle_data(X, Y):
    assert len(X) == len(Y)
    indices = np.random.permutation(len(X))
    X_shuffled = X[indices]
    Y_shuffled = Y[indices]
    return X_shuffled, Y_shuffled



#########################################################
###################### MLP MOdel ########################
#########################################################
def simple_mlp_model(input_size, num_classes, learning_rate=0.0001):
    # Designing input layer
    input_shape = (input_size,)
    inputs = Input(shape=input_shape)
    
    # Hidden layer with ReLU activation and L2 regularization
    x = Dense(128, kernel_initializer='he_uniform', activation='relu', kernel_regularizer=l2(0.01), name='fc1')(inputs)
    x = Dropout(0.2)(x)
    x = Dense(128, kernel_initializer='he_uniform', activation='relu', kernel_regularizer=l2(0.01), name='fc2')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, kernel_initializer='he_uniform', activation='relu', kernel_regularizer=l2(0.01), name='fc3')(x)
    x = Dropout(0.2)(x)
    # Output layer with softmax activation
    outputs = Dense(num_classes, activation='softmax', name='predictions')(x)
    
    # Create model
    model = Model(inputs, outputs)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def train(X,Y,coeff_samples):
    model=simple_mlp_model(input_size=coeff_samples,num_classes=2,learning_rate=0.0001)
    model.fit(X,Y,epochs=10, batch_size=512,validation_split=0.2,verbose=1)
    return model

In [21]:
'''align'''
# 确定每个切片起点
trig_file=np.load(r"F:\20241203_dilithium_y\profiling_trigger0.npy")
start_idx=trigger_idx(trig_file)
coeff_idx=[[], [], [], []]
for i in range(320):
    idx=i%5
    if idx!=4:
        coeff_idx[idx].append(start_idx[i])

# mod4 = 0 的205个samples
# mod4 = 0 的220个samples
# mod4 = 0 的220个samples
# mod4 = 0 的220个samples
coeff_samples=[205,220,220,220]


In [75]:
'''train'''
trace_url=r"F:\20241203_dilithium_y\profile\profiling_part{}.npy"
metadata_url=r"F:\金融密码杯\Dilithium\dilithium采集-比赛用-最终\训练集\metadata_profiling_part{}.npz"
models=[]
scalers=[]
for coeff_i in range(4):
    print("Coeff type:", coeff_i)
    X_m,Y_m = Reading_data_Train(trace_url,metadata_url,coeff_idx[coeff_i],coeff_samples,block_num=20,start=0,coeff_i=coeff_i)
    X_m,Y_m = shuffle_data(X_m, Y_m)
    scaler = StandardScaler()
    X_train_m, X_test_m, Y_train_m, Y_test_m = train_test_split(X_m, Y_m, test_size=0.2, random_state=42)
    scaler.fit(X_m)
    X_train_m = scaler.transform(X_m)
    scalers.append(scaler)
    Y_train_m = Y_m
    print("Preprocessed data shape:")
    print("X:",X_train_m.shape)
    print("Y:",Y_train_m.shape)
    m=train(X_train_m,Y_train_m,coeff_samples=coeff_samples[coeff_i])
    models.append(m)

Coeff type: 0


100%|██████████| 20/20 [00:15<00:00,  1.31it/s]


labels numbers:  [7356, 235395]
X: (242751, 205)
Y: (242751,)
Preprocessed data shape:
X: (242751, 205)
Y: (242751, 2)
Epoch 1/10
380/380 [==============================] - 3s 6ms/step - loss: 7.0068 - accuracy: 0.9352 - val_loss: 5.7990 - val_accuracy: 0.9698
Epoch 2/10
380/380 [==============================] - 2s 6ms/step - loss: 4.8207 - accuracy: 0.9729 - val_loss: 3.8818 - val_accuracy: 0.9768
Epoch 3/10
380/380 [==============================] - 2s 6ms/step - loss: 3.1627 - accuracy: 0.9791 - val_loss: 2.4904 - val_accuracy: 0.9841
Epoch 4/10
380/380 [==============================] - 2s 6ms/step - loss: 1.9987 - accuracy: 0.9838 - val_loss: 1.5473 - val_accuracy: 0.9863
Epoch 5/10
380/380 [==============================] - 2s 5ms/step - loss: 1.2282 - accuracy: 0.9869 - val_loss: 0.9389 - val_accuracy: 0.9883
Epoch 6/10
380/380 [==============================] - 2s 5ms/step - loss: 0.7428 - accuracy: 0.9884 - val_loss: 0.5642 - val_accuracy: 0.9899
Epoch 7/10
380/380 [=========

100%|██████████| 20/20 [00:13<00:00,  1.50it/s]


labels numbers:  [7596, 243075]
X: (250671, 220)
Y: (250671,)
Preprocessed data shape:
X: (250671, 220)
Y: (250671, 2)
Epoch 1/10
392/392 [==============================] - 3s 5ms/step - loss: 6.8197 - accuracy: 0.9430 - val_loss: 5.5781 - val_accuracy: 0.9696
Epoch 2/10
392/392 [==============================] - 2s 5ms/step - loss: 4.5689 - accuracy: 0.9713 - val_loss: 3.6161 - val_accuracy: 0.9732
Epoch 3/10
392/392 [==============================] - 2s 5ms/step - loss: 2.8968 - accuracy: 0.9795 - val_loss: 2.2361 - val_accuracy: 0.9871
Epoch 4/10
392/392 [==============================] - 2s 5ms/step - loss: 1.7643 - accuracy: 0.9864 - val_loss: 1.3369 - val_accuracy: 0.9895
Epoch 5/10
392/392 [==============================] - 2s 5ms/step - loss: 1.0435 - accuracy: 0.9888 - val_loss: 0.7797 - val_accuracy: 0.9918
Epoch 6/10
392/392 [==============================] - 2s 5ms/step - loss: 0.6079 - accuracy: 0.9903 - val_loss: 0.4525 - val_accuracy: 0.9929
Epoch 7/10
392/392 [=========

100%|██████████| 20/20 [00:13<00:00,  1.51it/s]


labels numbers:  [7452, 238465]
X: (245917, 220)
Y: (245917,)
Preprocessed data shape:
X: (245917, 220)
Y: (245917, 2)
Epoch 1/10
385/385 [==============================] - 3s 5ms/step - loss: 6.4955 - accuracy: 0.9608 - val_loss: 5.0994 - val_accuracy: 0.9690
Epoch 2/10
385/385 [==============================] - 2s 5ms/step - loss: 4.0256 - accuracy: 0.9733 - val_loss: 3.0491 - val_accuracy: 0.9810
Epoch 3/10
385/385 [==============================] - 2s 5ms/step - loss: 2.3569 - accuracy: 0.9816 - val_loss: 1.7400 - val_accuracy: 0.9869
Epoch 4/10
385/385 [==============================] - 2s 5ms/step - loss: 1.3221 - accuracy: 0.9867 - val_loss: 0.9548 - val_accuracy: 0.9911
Epoch 5/10
385/385 [==============================] - 2s 5ms/step - loss: 0.7186 - accuracy: 0.9891 - val_loss: 0.5127 - val_accuracy: 0.9899
Epoch 6/10
385/385 [==============================] - 2s 5ms/step - loss: 0.3865 - accuracy: 0.9900 - val_loss: 0.2759 - val_accuracy: 0.9920
Epoch 7/10
385/385 [=========

100%|██████████| 20/20 [00:13<00:00,  1.44it/s]


labels numbers:  [7599, 243170]
X: (250769, 220)
Y: (250769,)
Preprocessed data shape:
X: (250769, 220)
Y: (250769, 2)
Epoch 1/10
392/392 [==============================] - 3s 5ms/step - loss: 7.0196 - accuracy: 0.9327 - val_loss: 5.8603 - val_accuracy: 0.9705
Epoch 2/10
392/392 [==============================] - 2s 5ms/step - loss: 4.9146 - accuracy: 0.9710 - val_loss: 3.9927 - val_accuracy: 0.9752
Epoch 3/10
392/392 [==============================] - 2s 5ms/step - loss: 3.2791 - accuracy: 0.9771 - val_loss: 2.6047 - val_accuracy: 0.9832
Epoch 4/10
392/392 [==============================] - 2s 5ms/step - loss: 2.1071 - accuracy: 0.9828 - val_loss: 1.6448 - val_accuracy: 0.9885
Epoch 5/10
392/392 [==============================] - 2s 5ms/step - loss: 1.3168 - accuracy: 0.9865 - val_loss: 1.0154 - val_accuracy: 0.9897
Epoch 6/10
392/392 [==============================] - 2s 6ms/step - loss: 0.8099 - accuracy: 0.9887 - val_loss: 0.6214 - val_accuracy: 0.9904
Epoch 7/10
392/392 [=========

In [99]:
'''Attack'''

Z_url=r"F:\金融密码杯\Dilithium\dilithium采集-比赛用-最终\Z_unpack\第0组\Z_unpack_part{}.npz"
trace_attack_url = r"F:\20241203_dilithium_y\attack\traces_for_attack_part{}.npy"
meta_attack_url = r"F:\金融密码杯\Dilithium\dilithium采集-比赛用-最终\攻击集\第0组\meta_data_part{}.npz"
Attack(trace_attack_url,meta_attack_url,Z_url,models,coeff_idx,coeff_samples,scalers=scalers,block_num=4,start=0)




100%|██████████| 500/500 [00:31<00:00, 15.64it/s]

Guess: 1724, Real: 2002, Right: 1714


In [100]:
res=np.load(r"F:\20241203_dilithium_y\result.npy")
print(res)

[[  0   0 122]
 [  0   1 152]
 [  0   2 226]
 ...
 [  3 495 209]
 [  3 497 234]
 [  3 498  57]]
